# Using scaling tool in global model with LED scenario

In [1]:
import pandas as pd
import ixmp
import message_ix
import os

from message_ix.tools.lp_diag import LPdiag
from message_ix.tools.make_scaler import make_scaler
from message_ix.tools.make_scaler import show_range
from message_ix.tools.make_scaler import get_scaler_args

from message_ix.util import make_df

%matplotlib inline

lp = LPdiag()
mp = ixmp.Platform()

## Making a clone of the existing scenario 'baseline'

## Time to Solve the Model

Solution time: 48.14s

Using the unscaled scenario above, we make scaler arguments to solve scenario with scaled matrix coefficients

In [3]:
ssp ="LED"
model_name = f"ScalingSSP_{ssp}_v1.0"
scenario_name = f"baseline_DEFAULT"
scen = message_ix.Scenario(mp, model=model_name, scenario=scenario_name)


Existing index sets of 'EMISSION_EQUIVALENCE' ['node', 'emission', 'type_tec', 'year'] do not match []


In [4]:
current_directory = os.getcwd()
mps_dir = os.path.join(current_directory, f"{model_name}_{scenario_name}.mps")
scale_df = make_scaler(mps_dir, scen, bounds=4, steps=1)


Reading MPS-format file C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\tools\make_scaler\ScalingSSP_LED_v1.0_baseline_DEFAULT.mps.
Next section found: NAME          gamsmodel (line 1).
	Problem name: gamsmodel.
Next section found: ROWS (line 2).
	Row _obj (row_seq = 0) is the objective (goal function) row.
Next section found: COLUMNS (line 1197606).
Next section found: RHS (line 6989709).
	Id of RHS: rhs
Next section found: BOUNDS (line 7055589).
	Id of BOUNDS: bnd
Next section found: ENDATA (line 7559867).

Finished processing 7559867 lines of the MPS file: C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\tools\make_scaler\ScalingSSP_LED_v1.0_baseline_DEFAULT.mps.
LP has: 1197603 rows, 1104554 cols, 5792102 non-zeros, matrix density = 4.38e-06.
Numbers of redefined: RHS = 65879, ranges = 0, bounds = 504277.

The GF (objective) row named "_obj" has 169 elements.
Distribution of the GF (objective) values:
count    169.000000
mean       0.047908
std  

In [9]:
message_loc = "C:/Users/pratama/Documents/GitHub/MESSAGEix/"
file_name = f"ScalingSSP_{ssp}_v1.0_baseline_DEFAULT"
file_loc = f"{message_loc}/message_ix/message_ix/model/scaler/MsgScaler_{file_name}.gms"

In [10]:
import re

regex = re.compile(r'^([^&]*)(&)([^&]*)(&)([^&]*)')

with open(file_loc, 'r+') as f:
    old = f.readlines() # Pull the file contents to a list
    f.seek(0)           # Jump to start, so we overwrite instead of appending
    for line in old:
        f.write(line.replace("Fertilizer_Use|Nitrogen","'Fertilizer Use|Nitrogen'"))

## Run scenarios with scaled matrix

Results: solution time 57.8s

In [ ]:
model = "GENIE_sandbox"

base = message_ix.Scenario(mp, model=model, scenario="cb1000_splitCO2transdisp_unscaled")


sc_name = "cb1000_splitCO2transdisp_scaled"
sc = base.clone(
    model,
    sc_name,
    "introducing scaling tool for global model daccs scenario",
    keep_solution=False,
)
sc.check_out()

sc.commit(comment="Scaling tool test for global model")
sc.set_as_default()

In [ ]:
current_directory = os.getcwd()
mps_dir = os.path.join(current_directory, f"{model}_{sc_name}.mps")

In [ ]:
argument = get_scaler_args(scen)

sc.solve(solve_options={"writemps": mps_dir},
         gams_args=[argument]
        )

In [ ]:
sc.var("OBJ")["lvl"]

In [ ]:
scen.var("OBJ")["lvl"]

**Let's compare with running scenario without special arguments**

Results: solution time 50s

<code>

Primal simplex solved model.

Iteration:  4009    Objective     =       2609317.507987
Elapsed time = 40.50 sec. (46961.19 ticks, 4009 iterations)
Iteration:  4096    Objective     =       2609317.507987
Iteration:  5667    Objective     =       2609317.519581
Total crossover time = 29.84 sec. (24634.88 ticks)

Total time on 4 threads = 50.02 sec. (52935.76 ticks)

--- LP status (5): optimal with unscaled infeasibilities.
--- Cplex Time: 50.03sec (det. 52937.85 ticks)

Optimal solution found, but with infeasibilities after unscaling
Simplex iterations after crossover: 0
Objective:      2609317.507987

</code>

**Scaled with default GAMS options:**

Solution time: 119.23s

In [ ]:
import pandas as pd
import ixmp
import message_ix
import os

from message_ix.tools.lp_diag import LPdiag
from message_ix.tools.make_scaler import make_scaler
from message_ix.tools.make_scaler import show_range
from message_ix.tools.make_scaler import get_scaler_args

from message_ix.util import make_df

%matplotlib inline

lp = LPdiag()
mp = ixmp.Platform()

model = "GENIE_sandbox"

base = message_ix.Scenario(mp, model=model, scenario="cb1000_splitCO2transdisp_unscaled")


scen_name = "cb1000_splitCO2transdisp_scaled"
scen = base.clone(
    model,
    scen_name,
    "unscaled daccs scenario with split infrastructure",
    keep_solution=False,
)
scen.check_out()


argument = get_scaler_args(base)


scen.solve(solve_options={"advind": "determined by GAMS Bratio",
                          "epopt": "1.0e-06",
                          "lpmethod": "0",
                         },
           gams_args=[argument])
scen.var("OBJ")["lvl"]

**Unscaled with default GAMS settings**

Solution time: 93.52sec

In [ ]:
model = "GENIE_sandbox"

base = message_ix.Scenario(mp, model=model, scenario="cb1000_splitCO2transdisp")


scen_name = "cb1000_splitCO2transdisp_unscaled"
scen = base.clone(
    model,
    scen_name,
    "unscaled daccs scenario with split infrastructure",
    keep_solution=False,
)
scen.check_out()

scen.solve(solve_options={"advind": "determined by GAMS Bratio",
                          "epopt": "1.0e-06",
                          "lpmethod": "0",
                         })
scen.var("OBJ")["lvl"]

**Scaled with dual simplex:**

Solution time: 5152.6 s

In [ ]:
import pandas as pd
import ixmp
import message_ix
import os

from message_ix.tools.lp_diag import LPdiag
from message_ix.tools.make_scaler import make_scaler
from message_ix.tools.make_scaler import show_range
from message_ix.tools.make_scaler import get_scaler_args

from message_ix.util import make_df

%matplotlib inline

lp = LPdiag()
mp = ixmp.Platform()

model = "GENIE_sandbox"

base = message_ix.Scenario(mp, model=model, scenario="cb1000_splitCO2transdisp_unscaled")


scen_name = "cb1000_splitCO2transdisp_scaled"
scen = base.clone(
    model,
    scen_name,
    "unscaled daccs scenario with split infrastructure",
    keep_solution=False,
)
scen.check_out()


argument = get_scaler_args(base)


scen.solve(solve_options={"advind": "determined by GAMS Bratio",
                          "epopt": "1.0e-06",
                          "lpmethod": "2",
                         },
           gams_args=[argument])
scen.var("OBJ")["lvl"]

**Scaled with primal simplex:**

Solution time: 54665.56 s

In [ ]:
import pandas as pd
import ixmp
import message_ix
import os

from message_ix.tools.lp_diag import LPdiag
from message_ix.tools.make_scaler import make_scaler
from message_ix.tools.make_scaler import show_range
from message_ix.tools.make_scaler import get_scaler_args

from message_ix.util import make_df

%matplotlib inline

lp = LPdiag()
mp = ixmp.Platform()

model = "GENIE_sandbox"

base = message_ix.Scenario(mp, model=model, scenario="cb1000_splitCO2transdisp_unscaled")


scen_name = "cb1000_splitCO2transdisp_scaled"
scen = base.clone(
    model,
    scen_name,
    "unscaled daccs scenario with split infrastructure",
    keep_solution=False,
)
scen.check_out()


argument = get_scaler_args(base)


scen.solve(solve_options={"advind": "determined by GAMS Bratio",
                          "epopt": "1.0e-06",
                          "lpmethod": "1",
                         },
           gams_args=[argument])
scen.var("OBJ")["lvl"]

## Close the connection to the database

In [ ]:
mp.close_db()